In [ ]:
# Here I completely overhauled all my previous work, and reimported all the Verne data from a new source - much better!

In [33]:
import ebooklib
import nltk
import os
import numpy as np
import csv
import re

from ebooklib import epub

import matplotlib.pyplot as plt
from sklearn.metrics import pairwise
from sklearn.manifold import MDS

In [5]:
def epub2thtml(epub_path):
    book = epub.read_epub(epub_path)
    chapters = []
    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            chapters.append(item.get_content())
    return chapters

In [7]:
chapter = epub2thtml(completepath)

In [8]:
chapter

[b'<?xml version=\'1.0\' encoding=\'utf-8\'?>\n<!DOCTYPE html>\n<html xmlns="http://www.w3.org/1999/xhtml" xmlns:epub="http://www.idpf.org/2007/ops" epub:prefix="z3998: http://www.daisy.org/z3998/2012/vocab/structure/#" lang="en" xml:lang="en">\n  <head/>\n  <body><div class="DEFAULT_PAGE" id="d5e65">&#13;\n<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" height="100%" version="1.1" viewbox="0 0 600 800" width="100%">&#13;\n<rect fill="#000007" height="100%" width="600" x="0" y="0"/>&#13;\n<image height="800" transform="translate(0 0)" width="600" xlink:href="../images/cover.jpg"/>&#13;\n</svg>&#13;\n</div>&#13;\n</body>\n</html>\n',
 b'<?xml version=\'1.0\' encoding=\'utf-8\'?>\n<!DOCTYPE html>\n<html xmlns="http://www.w3.org/1999/xhtml" xmlns:epub="http://www.idpf.org/2007/ops" epub:prefix="z3998: http://www.daisy.org/z3998/2012/vocab/structure/#" lang="en" xml:lang="en">\n  <head/>\n  <body><div class="MAIN_PAGE" id="d5e70">&#13;\n<p class="h1-0001"

In [9]:
from bs4 import BeautifulSoup


In [10]:
blacklist = [   '[document]',   'noscript', 'header',   'html', 'meta', 'head','input', 'script',   ]

In [11]:
def chap2text(chap):
    output = ''
    soup = BeautifulSoup(chap, 'html.parser')
    text = soup.find_all(text=True)
    for t in text:
        if t.parent.name not in blacklist:
            output += '{} '.format(t)
    return output

In [12]:
def thtml2ttext(thtml):
    Output = []
    for html in thtml:
        text =  chap2text(html)
        Output.append(text)
    return Output

In [13]:
def epub2text(epub_path):
    chapters = epub2thtml(epub_path)
    ttext = thtml2ttext(chapters)
    return ttext

In [21]:
meta_path = 'C:\\Users\\kayde\\Desktop\\ENGL 343\\ENGLISH_TEXTS\\'
filepaths = []

for root, dirs, files in os.walk(meta_path):
    for file in files:
        filepaths.append(os.path.join(root, file))
        
len(filepaths), print(filepaths)

['C:\\Users\\kayde\\Desktop\\ENGL 343\\ENGLISH_TEXTS\\ADVENTURES OF CAPTAIN HATTERAS, THE - Jules Verne.epub', 'C:\\Users\\kayde\\Desktop\\ENGL 343\\ENGLISH_TEXTS\\ADVENTURES OF THREE ENGLISHMEN, THE - Jules Verne.epub', 'C:\\Users\\kayde\\Desktop\\ENGL 343\\ENGLISH_TEXTS\\ARCHIPELAGO ON FIRE, THE - Jules Verne.epub', 'C:\\Users\\kayde\\Desktop\\ENGL 343\\ENGLISH_TEXTS\\AROUND THE MOON - Jules Verne.epub', 'C:\\Users\\kayde\\Desktop\\ENGL 343\\ENGLISH_TEXTS\\AROUND THE WORLD IN EIGHTY - Jules Verne.epub', "C:\\Users\\kayde\\Desktop\\ENGL 343\\ENGLISH_TEXTS\\BEGUM'S FORTUNE, THE - Jules Verne.epub", 'C:\\Users\\kayde\\Desktop\\ENGL 343\\ENGLISH_TEXTS\\CARPATHIAN CASTLE, THE - Jules Verne.epub', 'C:\\Users\\kayde\\Desktop\\ENGL 343\\ENGLISH_TEXTS\\CESAR CASCABEL - Jules Verne.epub', 'C:\\Users\\kayde\\Desktop\\ENGL 343\\ENGLISH_TEXTS\\CHILDREN OF CAPTAIN GRANT, THE - Jules Verne.epub', 'C:\\Users\\kayde\\Desktop\\ENGL 343\\ENGLISH_TEXTS\\CLAUDIUS BOMBARNAC - Jules Verne.epub', 'C:\\Users

(47, None)

In [28]:
texts = []
for file in filepaths:
    out = epub2text(file)
    texts.append(out)


In [45]:
string_texts = [' '.join([str(c) for c in lst]) for lst in texts]

In [46]:
import pandas as pd

date = [f[44:48] for f in files]
df = pd.DataFrame(files, columns=['FILE_PATH'])
df['TEXT'] = string_texts


In [47]:
df

,FILE_PATH,TEXT
0,"ADVENTURES OF CAPTAIN HATTERAS, THE - Jules Ve...",\n \n \n \n \n \n \n THE ADVENTURES OF CAPTAI...
1,"ADVENTURES OF THREE ENGLISHMEN, THE - Jules Ve...",\n \n \n \n \n \n \n THE ADVENTURES OF THREE ...
2,"ARCHIPELAGO ON FIRE, THE - Jules Verne.epub",\n \n \n \n \n \n \n THE ARCHIPELAGO ON FIRE ...
3,AROUND THE MOON - Jules Verne.epub,\n \n \n \n \n \n \n AROUND THE MOON \n \n ...
4,AROUND THE WORLD IN EIGHTY - Jules Verne.epub,\n \n \n \n \n \n \n AROUND THE WORLD IN EIGH...
5,"BEGUM'S FORTUNE, THE - Jules Verne.epub",\n \n \n \n \n \n \n THE BEGUM’S FORTUNE \n ...
6,"CARPATHIAN CASTLE, THE - Jules Verne.epub",\n \n \n \n \n \n \n T HE CARPATHIAN CASTLE \...
7,CESAR CASCABEL - Jules Verne.epub,\n \n \n \n \n \n \n CÉSAR CASCABEL \n \n T...
8,"CHILDREN OF CAPTAIN GRANT, THE - Jules Verne.epub",\n \n \n \n \n \n \n THE CHILDREN OF CAPTAIN ...
9,CLAUDIUS BOMBARNAC - Jules Verne.epub,\n \n \n \n \n \n \n CLAUDIUS BOMBARNAC \n ...


In [34]:
def my_cleaner(s):
    cleaned = re.sub('<[^>]+>', '', s) # Removes stuff between <>
    cleaned = re.sub('\xa0', ' ', cleaned) # Replaces '\xa0' characters with space
    cleaned = re.sub('\n', ' ', cleaned) # Replaces newline characters with space
    cleaned = re.sub('(\w+)?@\w+','',cleaned) # Removes whole word if it has an '@' (i.e. usernames)
    cleaned = re.sub('[^a-z\s\.\d\']+','',cleaned,flags=re.IGNORECASE) #Removes other non-alphabetical characters(EXCEPT PERIODS)
    cleaned = re.sub('(\s+)',' ',cleaned) # Remove multiple spaces
    cleaned = re.sub('(\s\.)', '.', cleaned) # Replaces space then period ' .' with period
    cleaned = re.sub('(\.\.)', '.', cleaned) # Fixes double periods
    cleaned_str=cleaned.lower()
    
    return cleaned_str

In [57]:
clean_texts = []
for text in string_texts:
    clean = my_cleaner(text)
    clean_texts.append(clean)

In [64]:
df['CLEAN'] = clean_texts


In [65]:
df

,FILE_PATH,TEXT,CLEAN
0,"ADVENTURES OF CAPTAIN HATTERAS, THE - Jules Ve...",\n \n \n \n \n \n \n THE ADVENTURES OF CAPTAI...,the adventures of captain hatteras anonymous ...
1,"ADVENTURES OF THREE ENGLISHMEN, THE - Jules Ve...",\n \n \n \n \n \n \n THE ADVENTURES OF THREE ...,the adventures of three englishmen and three ...
2,"ARCHIPELAGO ON FIRE, THE - Jules Verne.epub",\n \n \n \n \n \n \n THE ARCHIPELAGO ON FIRE ...,the archipelago on fire anonymous translation...
3,AROUND THE MOON - Jules Verne.epub,\n \n \n \n \n \n \n AROUND THE MOON \n \n ...,around the moon translated by lewis page merc...
4,AROUND THE WORLD IN EIGHTY - Jules Verne.epub,\n \n \n \n \n \n \n AROUND THE WORLD IN EIGH...,around the world in eighty days anonymous tra...
5,"BEGUM'S FORTUNE, THE - Jules Verne.epub",\n \n \n \n \n \n \n THE BEGUM’S FORTUNE \n ...,the begums fortune translated by agnes kinloc...
6,"CARPATHIAN CASTLE, THE - Jules Verne.epub",\n \n \n \n \n \n \n T HE CARPATHIAN CASTLE \...,t he carpathian castle anonymous translation ...
7,CESAR CASCABEL - Jules Verne.epub,\n \n \n \n \n \n \n CÉSAR CASCABEL \n \n T...,csar cascabel translated by a. estoclet first...
8,"CHILDREN OF CAPTAIN GRANT, THE - Jules Verne.epub",\n \n \n \n \n \n \n THE CHILDREN OF CAPTAIN ...,the children of captain grant anonymous trans...
9,CLAUDIUS BOMBARNAC - Jules Verne.epub,\n \n \n \n \n \n \n CLAUDIUS BOMBARNAC \n ...,claudius bombarnac anonymous translation 1894...


In [49]:
import requests
import re
def americanize(string):
    url ="https://raw.githubusercontent.com/hyperreality/American-British-English-Translator/master/data/british_spellings.json"
    british_to_american = requests.get(url).json()    
    for british_spelling, \american_spelling in british_to_american.items():
        #string = string.replace(british_spelling, american_spelling) 
        string = re.sub(f'(?<![a-zA-Z]){british_spelling}(?![a-z-Z])', american_spelling, string)
    return string

In [70]:
import timeit

start = timeit.default_timer()

american_texts = []
for text in clean_texts:
    clean = americanize(text)
    american_texts.append(clean)

stop = timeit.default_timer()

print('Time: ', stop - start) 

Time:  983.7920273


In [72]:
df['AMERICAN'] = american_texts

In [73]:
df

,FILE_PATH,TEXT,CLEAN,AMERICAN
0,"ADVENTURES OF CAPTAIN HATTERAS, THE - Jules Ve...",\n \n \n \n \n \n \n THE ADVENTURES OF CAPTAI...,the adventures of captain hatteras anonymous ...,the adventures of captain hatteras anonymous ...
1,"ADVENTURES OF THREE ENGLISHMEN, THE - Jules Ve...",\n \n \n \n \n \n \n THE ADVENTURES OF THREE ...,the adventures of three englishmen and three ...,the adventures of three englishmen and three ...
2,"ARCHIPELAGO ON FIRE, THE - Jules Verne.epub",\n \n \n \n \n \n \n THE ARCHIPELAGO ON FIRE ...,the archipelago on fire anonymous translation...,the archipelago on fire anonymous translation...
3,AROUND THE MOON - Jules Verne.epub,\n \n \n \n \n \n \n AROUND THE MOON \n \n ...,around the moon translated by lewis page merc...,around the moon translated by lewis page merc...
4,AROUND THE WORLD IN EIGHTY - Jules Verne.epub,\n \n \n \n \n \n \n AROUND THE WORLD IN EIGH...,around the world in eighty days anonymous tra...,around the world in eighty days anonymous tra...
5,"BEGUM'S FORTUNE, THE - Jules Verne.epub",\n \n \n \n \n \n \n THE BEGUM’S FORTUNE \n ...,the begums fortune translated by agnes kinloc...,the begums fortune translated by agnes kinloc...
6,"CARPATHIAN CASTLE, THE - Jules Verne.epub",\n \n \n \n \n \n \n T HE CARPATHIAN CASTLE \...,t he carpathian castle anonymous translation ...,t he carpathian castle anonymous translation ...
7,CESAR CASCABEL - Jules Verne.epub,\n \n \n \n \n \n \n CÉSAR CASCABEL \n \n T...,csar cascabel translated by a. estoclet first...,csar cascabel translated by a. estoclet first...
8,"CHILDREN OF CAPTAIN GRANT, THE - Jules Verne.epub",\n \n \n \n \n \n \n THE CHILDREN OF CAPTAIN ...,the children of captain grant anonymous trans...,the children of captain grant anonymous trans...
9,CLAUDIUS BOMBARNAC - Jules Verne.epub,\n \n \n \n \n \n \n CLAUDIUS BOMBARNAC \n ...,claudius bombarnac anonymous translation 1894...,claudius bombarnac anonymous translation 1894...


In [80]:
from sklearn.feature_extraction.text import CountVectorizer

# Build DTM

vectorizer = CountVectorizer(stop_words='english', min_df=2, encoding='utf8')
dtm = vectorizer.fit_transform(american_texts)
vocab = vectorizer.get_feature_names()
matrix = dtm.toarray()

In [81]:
DTM = pd.DataFrame(matrix, columns=vocab)
DTM.shape

(47, 26915)

In [82]:
final_df = pd.concat([df, DTM], axis=1)
final_df.shape

(47, 26919)

In [83]:
final_df
# Yay, it worked! Now it can be used for testing/analysis

,FILE_PATH,TEXT,CLEAN,AMERICAN,000,0f,0h,10,100,1000,...,zone,zones,zoological,zoologist,zoologists,zoology,zoophytes,zorn,zorns,zoroaster
0,"ADVENTURES OF CAPTAIN HATTERAS, THE - Jules Ve...",\n \n \n \n \n \n \n THE ADVENTURES OF CAPTAI...,the adventures of captain hatteras anonymous ...,the adventures of captain hatteras anonymous ...,0,0,0,3,2,0,...,4,2,1,0,0,0,0,0,0,0
1,"ADVENTURES OF THREE ENGLISHMEN, THE - Jules Ve...",\n \n \n \n \n \n \n THE ADVENTURES OF THREE ...,the adventures of three englishmen and three ...,the adventures of three englishmen and three ...,0,0,0,0,4,1,...,0,0,0,0,0,0,0,60,1,0
2,"ARCHIPELAGO ON FIRE, THE - Jules Verne.epub",\n \n \n \n \n \n \n THE ARCHIPELAGO ON FIRE ...,the archipelago on fire anonymous translation...,the archipelago on fire anonymous translation...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,AROUND THE MOON - Jules Verne.epub,\n \n \n \n \n \n \n AROUND THE MOON \n \n ...,around the moon translated by lewis page merc...,around the moon translated by lewis page merc...,0,0,0,0,2,1,...,0,0,0,0,0,0,0,0,0,0
4,AROUND THE WORLD IN EIGHTY - Jules Verne.epub,\n \n \n \n \n \n \n AROUND THE WORLD IN EIGH...,around the world in eighty days anonymous tra...,around the world in eighty days anonymous tra...,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
5,"BEGUM'S FORTUNE, THE - Jules Verne.epub",\n \n \n \n \n \n \n THE BEGUM’S FORTUNE \n ...,the begums fortune translated by agnes kinloc...,the begums fortune translated by agnes kinloc...,0,0,0,1,0,0,...,3,0,0,0,0,0,0,0,0,0
6,"CARPATHIAN CASTLE, THE - Jules Verne.epub",\n \n \n \n \n \n \n T HE CARPATHIAN CASTLE \...,t he carpathian castle anonymous translation ...,t he carpathian castle anonymous translation ...,0,0,0,0,0,0,...,2,1,0,0,0,0,0,0,0,0
7,CESAR CASCABEL - Jules Verne.epub,\n \n \n \n \n \n \n CÉSAR CASCABEL \n \n T...,csar cascabel translated by a. estoclet first...,csar cascabel translated by a. estoclet first...,0,0,0,1,0,0,...,3,3,0,0,0,0,0,0,0,0
8,"CHILDREN OF CAPTAIN GRANT, THE - Jules Verne.epub",\n \n \n \n \n \n \n THE CHILDREN OF CAPTAIN ...,the children of captain grant anonymous trans...,the children of captain grant anonymous trans...,1,0,0,3,3,1,...,5,2,2,0,0,0,0,0,0,0
9,CLAUDIUS BOMBARNAC - Jules Verne.epub,\n \n \n \n \n \n \n CLAUDIUS BOMBARNAC \n ...,claudius bombarnac anonymous translation 1894...,claudius bombarnac anonymous translation 1894...,0,0,0,2,0,0,...,3,4,0,0,0,0,0,0,0,1
